# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f30280d6340>
├── 'a' --> tensor([[-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495]])
└── 'x' --> <FastTreeValue 0x7f3028137e80>
    └── 'c' --> tensor([[-0.9004,  0.1704, -1.2218,  1.7708],
                        [-1.6572,  0.0358,  0.0639, -0.2778],
                        [-0.0517, -2.1045,  0.0235, -1.0484]])

In [4]:
t.a

tensor([[-0.5207, -0.7976, -0.4328],
        [ 0.3064, -1.6373, -1.5495]])

In [5]:
%timeit t.a

63.8 ns ± 0.0494 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f30280d6340>
├── 'a' --> tensor([[-0.2176,  0.7201, -0.7425],
│                   [-0.3279,  1.2687,  0.7329]])
└── 'x' --> <FastTreeValue 0x7f3028137e80>
    └── 'c' --> tensor([[-0.9004,  0.1704, -1.2218,  1.7708],
                        [-1.6572,  0.0358,  0.0639, -0.2778],
                        [-0.0517, -2.1045,  0.0235, -1.0484]])

In [7]:
%timeit t.a = new_value

67.3 ns ± 0.0772 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5207, -0.7976, -0.4328],
               [ 0.3064, -1.6373, -1.5495]]),
    x: Batch(
           c: tensor([[-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484]]),
       ),
)

In [10]:
b.a

tensor([[-0.5207, -0.7976, -0.4328],
        [ 0.3064, -1.6373, -1.5495]])

In [11]:
%timeit b.a

58.6 ns ± 0.0415 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0027,  0.6656, -1.9591],
               [ 0.2981, -0.7660, -0.4932]]),
    x: Batch(
           c: tensor([[-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 0.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

815 ns ± 0.227 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 24.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 819 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2f849de460>
├── 'a' --> tensor([[[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]],
│           
│                   [[-0.5207, -0.7976, -0.4328],
│                    [ 0.3064, -1.6373, -1.5495]]])
└── 'x' --> <FastTreeValue 0x7f2f849ded30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2f84c152b0>
├── 'a' --> tensor([[-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495],
│                   [-0.5207, -0.7976, -0.4328],
│                   [ 0.3064, -1.6373, -1.5495]])
└── 'x' --> <FastTreeValue 0x7f2f84e38eb0>
    └── 'c' --> tensor([[-0.9004,  0.1704, -1.2218,  1.7708],
                        [-1.6572,  0.0358,  0.0639, -0.2778],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 92.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

54.6 µs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.9004,  0.1704, -1.2218,  1.7708],
                       [-1.6572,  0.0358,  0.0639, -0.2778],
                       [-0.0517, -2.1045,  0.0235, -1.0484]],
              
                      [[-0.9004,  0.1704, -1.2218,  1.7708],
                       [-1.6572,  0.0358,  0.0639, -0.2778],
                       [-0.0517, -2.1045,  0.0235, -1.0484]],
              
                      [[-0.9004,  0.1704, -1.2218,  1.7708],
                       [-1.6572,  0.0358,  0.0639, -0.2778],
                       [-0.0517, -2.1045,  0.0235, -1.0484]],
              
                      [[-0.9004,  0.1704, -1.2218,  1.7708],
                       [-1.6572,  0.0358,  0.0639, -0.2778],
                       [-0.0517, -2.1045,  0.0235, -1.0484]],
              
                      [[-0.9004,  0.1704, -1.2218,  1.7708],
                       [-1.6572,  0.0358,  0.0639, -0.2778],
                       [-0.0517, -2.1045,  0.0235, -1.0484]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484],
                      [-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484],
                      [-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484],
                      [-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484],
                      [-0.9004,  0.1704, -1.2218,  1.7708],
                      [-1.6572,  0.0358,  0.0639, -0.2778],
                      [-0.0517, -2.1045,  0.0235, -1.0484],
                      [-0.9004,  0.1704, -1.2218,  1.7708],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 981 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

326 µs ± 4.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
